In [1]:
!pip install pandas==1.5.3
!pip install selenium
!pip install undetected_chromedriver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 21.6 MB/s eta 0:00:00m eta 0:00:010:01:01
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached trio-0.22.0-py3-none-any.whl (384 kB)
  Using cached exceptiongroup-1.1.0-py3-none-any.whl (14 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00
  Using cached undetected-chromedriver-3.4.6.tar.gz (61 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.7 MB/s eta 0:00:00
  Created wheel for undetected_chromedriver: filename=undetected_chromedriver-3.4.6-py3-none-any.whl size=44484 sha256=d197518893a359009db8dee0fbb6b0ed38cd1f2e94517cdf8cf4afaaf6a3df54
  Stored in directory: /Users/vtamprateep/Library/Caches/pip/wheels/59/91/0c/e10481bbb66f4c4b09cc2aa1356320

In [1]:
from selenium.common.exceptions import WebDriverException, NoSuchElementException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.chromium.options import ChromiumOptions

import pandas as pd
import undetected_chromedriver as webdriver
import json
import random
import time
import csv

## Part 1: Scrape search results

In [18]:
# Get sample list of artists to search and get their web pages
artist_list = pd.read_csv("seed_artist.csv", header = None)

In [21]:
artist_list[1001:]

,0
1001,Paul Oakenfold
1002,Steely Dan
1003,Kenny Loggins
1004,Daughter
1005,Andy Williams
...,...
1517,Ying Yang Twins
1518,Great Lake Swimmers
1519,Zac Brown Band
1520,Procol Harum


In [2]:
# Configure options to disable javascript
options = webdriver.ChromeOptions()
options.add_experimental_option( "prefs",{'profile.managed_default_content_settings.javascript': 2})

# Configure load strategy to eager to speed up load time
options.page_load_strategy = "eager"

In [22]:
# Initialize webdriver
driver = webdriver.Chrome(options)

In [23]:
# Search artist name on website, pull URL to first search result
# Write page URL to file every 50 artists pulled in case bot detection kicks in

base_df = pd.DataFrame()
batch = 0
first_write = True
file_name = "artist_search_500.csv"

for artist in artist_list[0][1001:]:
    artist_format = artist.replace(" ", "+")
    search_query = f"https://www.concertarchives.org/bands?search={artist_format}"
    driver.get(search_query)
    
    # Scan page and get search results in df
    try:
        table_element = driver.find_element(By.CLASS_NAME, "table-responsive")
        table_html = table_element.get_attribute("outerHTML")
        artist_df = pd.read_html(table_html, extract_links = "body")[0]
    except:
        continue
    
    # Append artist name, add to dataframe, increment count
    artist_df["Name"] = artist
    
    if base_df.empty:
        base_df = artist_df.copy()
    else:
        base_df = pd.concat([base_df, artist_df])
        
    batch += 1

    # Write to file every 50 artists
    if batch == 50 and first_write:  # Write to file every 50 artists
        base_df.to_csv(file_name, index = False)
        batch = 0
        base_df = pd.DataFrame()
        first_write = False
    elif batch == 50:
        base_df.to_csv(file_name, index = False, mode = "a", header = False)
        batch = 0
        base_df = pd.DataFrame()
        
    time.sleep(random.random() / 2 + 0.5)  # Randomly wait 0.5 - 1.5 seconds before next artist
    
# Save remaining scraped band page URLs to file
if not base_df.empty:
    base_df.to_csv(file_name, index = False, mode = "a", header = False)

## Part 2: Get top search result by edit distance

In [53]:
!pip install levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 1.8 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.3 MB/s eta 0:00:004.6 MB/s eta 0:00:01


In [24]:
import re
import Levenshtein

In [26]:
# Import and format data
search_df = pd.read_csv("artist_search_500.csv", names = ["search_name", "concert_count", "artist"])
search_df.head()

,search_name,concert_count,artist
0,"('Paul Oakenfold', '/bands/paul-oakenfold')","('569 concerts', None)",Paul Oakenfold
1,"('DJ Paul Oakenfold', '/bands/dj-paul-oakenfold')","('8 concerts', None)",Paul Oakenfold
2,"('Perfecto Featuring Paul Oakenfold', '/bands/...","('1 concerts', None)",Paul Oakenfold
3,"('Steely Dan', '/bands/steely-dan')","('1,021 concerts', None)",Steely Dan
4,"('Steely Dane', '/bands/steely-dane')","('3 concerts', None)",Steely Dan


In [27]:
# Split tuple columns for name and href
search_results_name = search_df["search_name"].tolist()

for i in range(len(search_results_name)):
    search_results_name[i] = re.sub(r"[\(\)'']", "", search_results_name[i])
    search_results_name[i] = search_results_name[i].split(", /")
    
results_name_df = pd.DataFrame(search_results_name)

# Extract count of concerts
search_concert_count = search_df["concert_count"].tolist()

for i in range(len(search_concert_count)):
    search_concert_count[i] = re.sub(r"[\(\)'',]", "", search_concert_count[i])
    search_concert_count[i] = search_concert_count[i].split("concerts")[0].strip()
    
results_concert_df = pd.DataFrame(search_concert_count)
results_concert_df[0] = results_concert_df[0].astype('int')

In [28]:
# Recombine into cleaned dataframe
cleaned_search_df = pd.concat([search_df, results_name_df, results_concert_df], axis = 1)
cleaned_search_df = cleaned_search_df.drop(columns = ["search_name", "concert_count"])
cleaned_search_df.columns = ["artist", "search_result", "page_url", "concert_count"]

In [29]:
# Calculate levenshtein distance
cleaned_search_df["l_score"] = pd.NA
for i, r in cleaned_search_df.iterrows():
    cleaned_search_df["l_score"][i] = Levenshtein.distance(r["artist"], r["search_result"])
    
cleaned_search_df.head()

/var/folders/d3/k0v5vst508sc7glyv2sbz9880000gn/T/ipykernel_3749/1665845050.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_search_df["l_score"][i] = Levenshtein.distance(r["artist"], r["search_result"])


,artist,search_result,page_url,concert_count,l_score
0,Paul Oakenfold,Paul Oakenfold,bands/paul-oakenfold,569,0
1,Paul Oakenfold,DJ Paul Oakenfold,bands/dj-paul-oakenfold,8,3
2,Paul Oakenfold,Perfecto Featuring Paul Oakenfold,bands/perfecto-featuring-paul-oakenfold,1,19
3,Steely Dan,Steely Dan,bands/steely-dan,1021,0
4,Steely Dan,Steely Dane,bands/steely-dane,3,1


In [30]:
# Dense rank and pull highest rank result per artist
ranked_series = cleaned_search_df.groupby("artist")["l_score"].rank(method="dense", ascending=True)
ranked_series.name = "rank"
ranked_search_df = pd.concat([cleaned_search_df, ranked_series], axis = 1)
ranked_name_df = ranked_search_df[ranked_search_df["rank"] == 1]

ranked_name_df

,artist,search_result,page_url,concert_count,l_score,rank
0,Paul Oakenfold,Paul Oakenfold,bands/paul-oakenfold,569,0,1.0
3,Steely Dan,Steely Dan,bands/steely-dan,1021,0,1.0
28,Kenny Loggins,Kenny Loggins,bands/kenny-loggins,851,0,1.0
30,Daughter,Daughter,bands/daughter,413,0,1.0
56,Andy Williams,andy williams,bands/andy-williams,80,2,1.0
...,...,...,...,...,...,...
4033,Ying Yang Twins,Ying Yang Twins,bands/ying-yang-twins,300,0,1.0
4035,Great Lake Swimmers,Great Lake Swimmers,bands/great-lake-swimmers,695,0,1.0
4038,Zac Brown Band,Zac Brown Band,bands/zac-brown-band,1272,0,1.0
4040,Procol Harum,Procol Harum,bands/procol-harum,283,0,1.0


In [31]:
# Dense rank on concert count, pull highest rank result
ranked_concerts = cleaned_search_df.groupby("artist")["concert_count"].rank(method="dense", ascending=False)
ranked_concerts.name = "rank"
ranked_concerts_df = ranked_name_df.copy()
ranked_concerts_df["rank"] = ranked_concerts
ranked_concerts_df = ranked_concerts_df[ranked_concerts_df["rank"] == 1]

ranked_concerts_df

,artist,search_result,page_url,concert_count,l_score,rank
0,Paul Oakenfold,Paul Oakenfold,bands/paul-oakenfold,569,0,1.0
3,Steely Dan,Steely Dan,bands/steely-dan,1021,0,1.0
28,Kenny Loggins,Kenny Loggins,bands/kenny-loggins,851,0,1.0
30,Daughter,Daughter,bands/daughter,413,0,1.0
56,Andy Williams,andy williams,bands/andy-williams,80,2,1.0
...,...,...,...,...,...,...
4033,Ying Yang Twins,Ying Yang Twins,bands/ying-yang-twins,300,0,1.0
4035,Great Lake Swimmers,Great Lake Swimmers,bands/great-lake-swimmers,695,0,1.0
4038,Zac Brown Band,Zac Brown Band,bands/zac-brown-band,1272,0,1.0
4040,Procol Harum,Procol Harum,bands/procol-harum,283,0,1.0


In [32]:
# Save to csv
ranked_concerts_df.to_csv("fuzzy_match_artist_500.csv")

## Part 3: Scrape concert page per band

In [6]:
# Import list of band page URLs
artist_page_list = pd.read_csv("fuzzy_match_artist.csv")
artist_page_list[344:].head()

,Unnamed: 0,artist,search_result,page_url,concert_count,l_score,rank
344,4451,Faithless,Faithless,bands/faithless,291,0,1
345,4462,Alice Cooper,Alice Cooper,bands/alice-cooper,2950,0,1
346,4486,P.O.D.,P.O.D.,bands/p-o-d,940,0,1
347,4507,Slayer,Slayer,bands/slayer,1760,0,1
348,4532,Fiona Apple,Fiona Apple,bands/fiona-apple,285,0,1


In [7]:
# Initialize webdriver again if you took a break between getting
# artist URL and scraping artist page

driver = webdriver.Chrome(options)

In [8]:
# Loop through artists, get concerts info
base_df = pd.DataFrame()
base_url = "https://www.concertarchives.org/"
batch = 0
first_write = True
file_name = "artist_concert_data_344.csv"

for i, r in artist_page_list[344:].iterrows():
    # Load page and get table HTML
    
    for page in range(1, 6):
        query = f"{base_url}{r['page_url']}?page={page}"
        try:
            driver.get(query)
            table_element = driver.find_element(By.ID, "band-show-table-condensed")
            table_html = table_element.get_attribute("outerHTML")

            # Pass to Pandas, condense to one data frame
            df_list = pd.read_html(table_html)
            df = df_list[0]

            for i in range(1,len(df_list)):
                next_df = df_list[i]
                next_df.columns = df.columns  # Format columns for concat
                df = pd.concat([df, next_df])
        except:
            break
        
        df["Artist"] = r["artist"]
        
        if base_df.empty:
            base_df = df.copy()
        else:
            base_df = pd.concat([base_df, df])
            
        time.sleep(random.random() / 4 + 0.25)
        batch += 1
    
    # Write to file every 50 artists
    if batch >= 50 and first_write:  # Write to file every 50 artists
        base_df.to_csv(file_name, index = False)
        base_df = pd.DataFrame()
        batch = 0
        first_write = False
    elif batch >= 50:
        base_df.to_csv(file_name, index = False, mode = "a", header = False)
        batch = 0
        base_df = pd.DataFrame()
    
# Save remaining scraped band page URLs to file
if not base_df.empty:
    base_df.to_csv(file_name, index = False, mode = "a", header = False)

In [9]:
base_df
if not base_df.empty:
    base_df.to_csv(file_name, index = False, mode = "a", header = False)